# Installation des librairies 

In [1]:
!python3 -m spacy download fr_core_news_md
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 9.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 KB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 64.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 KB 59.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━

# Import des librairies

In [9]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import re
from textblob import TextBlob
import os
import glob
import matplotlib.pyplot as plt 
import seaborn as sns 

import matplotlib
from matplotlib import gridspec
from matplotlib import rcParams
#Paramètres graphiques 
rcParams['figure.figsize'] = 15,8 #Taille de la figure affichée
sns.set_style("darkgrid")#style de l'arrière plan de seaborn 
sns.set_palette("pastel")#Couleurs utilisées dans les graphiques 
matplotlib.rcParams["figure.dpi"] = 200 #dpi = dot per inch , résolution des graphiques 


pd.set_option('display.max_columns', None)

import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import WordPunctTokenizer

#Downloads et initialisation des variables : 
nltk.download('stopwords')
sw = stopwords.words('french')
stemmer= FrenchStemmer()
nlp= spacy.load("fr_core_news_md")
tokenizer=WordPunctTokenizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Fonctions

In [10]:
#Preprocessing
def check_nan(df):
    for i in df.columns.tolist():
        print("Valeurs nan dans "+str(i)+" : "+str(df[i].isna().sum()))

def check_unique(df):
    for i in df.columns.tolist():
        print("Valeurs uniques dans "+str(i)+" : "+str(df[i].nunique()))


def data_reading(df, cols_to_list, cols_to_string, cols_to_date) :
    for col in cols_to_list : 
        df[col] = df[col].apply(eval)
        
    for col in cols_to_string : 
        df[col]= df[col].astype(str)
        
    for col in cols_to_date : 
        df[col] = pd.to_datetime(df[col])
                
    return df 

In [11]:
#Renvoie le texte en entrée sans ponctuation
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with spaces, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans(string.punctuation + "’", ' '*(len(string.punctuation)+1))
    # return the text stripped of punctuation marks
    return text.translate(translator)


#Tokenization des commentaires :
def tokenize(text):
    return tokenizer.tokenize(text)

#On retire les stopwords : 
def remove_stopwords(text):
    output= [i for i in text if i not in sw]
    return output

#Lemmatisation  :
def lemmatizer(text):
    doc =nlp(" ".join(text))
    lemm_text = [word.lemma_ for word in doc]
    return lemm_text

#Stemmatisation : 
def stemmer_func(text) :
    return [stemmer.stem(word) for word in text]


#Preprocessing des avis pour le dataframe renseigné
def preprocessing_reviews_fr(df,review_column) : 
    print("-Nombre de caractères par avis.")
    df["nb_carac_fr"] = df[review_column].str.len()#Ajout du nombre de caractères 
    print("-Nombre de mots par avis.")
    df["nb_words_fr"] = df[review_column].str.split().str.len()#Ajout du nombre de mots 
    print("-On retire la ponctuation des avis.")
    df["review_no_punct_min_fr"] = df[review_column].apply(remove_punctuation).str.lower()#On enlève la ponctuation et on met le texte en minuscule
    print("-Tokenisation des avis.")
    df["tokenized_fr"] = df["review_no_punct_min_fr"].apply(tokenize)#Tokenization des avis sans ponctuation en minuscule
    print("-On retire les stop words français.")
    df["tokenized_no_sw_fr"] = df["tokenized_fr"].apply(remove_stopwords)
    print("-Lemmatisation française des avis tokenisés.")
    df["lemmatized_fr"] = df["tokenized_no_sw_fr"].apply(lemmatizer)
    print("-Stemmatisation française des avis tokenisés.")
    df["stemmed_fr"] = df["tokenized_no_sw_fr"].apply(stemmer_func)
    return df 


# Import des données

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
data_path="/content/drive/MyDrive/WebScrapping/data/"

In [35]:
df = pd.read_csv(data_path+"reviews_sc_all.csv")
print(df.shape)
check_nan(df)
print()
check_unique(df)
df.head(3)

(3368, 5)
Valeurs nan dans id_movie : 0
Valeurs nan dans title : 0
Valeurs nan dans contenu_complet : 0
Valeurs nan dans note : 0
Valeurs nan dans plateforme : 0

Valeurs uniques dans id_movie : 30
Valeurs uniques dans title : 30
Valeurs uniques dans contenu_complet : 3364
Valeurs uniques dans note : 20
Valeurs uniques dans plateforme : 2


,id_movie,title,contenu_complet,note,plateforme
0,292222,the innocents,L’enfance a toujours été un terreau particuliè...,3.5,senscritique
1,292222,the innocents,"Wow, voilà un film qui n'a pas volé ses récomp...",4.5,senscritique
2,292222,the innocents,Même s’il s’était essayé une première fois à l...,3.5,senscritique


# Preprocessing des données 

In [27]:
df["message_length"]=df['contenu_complet'].str.len()
print("Shape : ",df.shape)
df.head()

Shape :  (3368, 6)


,id_movie,title,contenu_complet,note,plateforme,message_length
0,292222,the innocents,L’enfance a toujours été un terreau particuliè...,3.5,senscritique,2885
1,292222,the innocents,"Wow, voilà un film qui n'a pas volé ses récomp...",4.5,senscritique,4843
2,292222,the innocents,Même s’il s’était essayé une première fois à l...,3.5,senscritique,5655
3,292222,the innocents,-Et voici donc ma palme d'or 2021 ! Le film a ...,4.5,senscritique,1178
4,292222,the innocents,"Eskil Vogt, réalisateur de Blind et scénariste...",3.5,senscritique,3261


In [28]:
df["nb_words"]=df['contenu_complet'].str.split().str.len()
df.head()

,id_movie,title,contenu_complet,note,plateforme,message_length,nb_words
0,292222,the innocents,L’enfance a toujours été un terreau particuliè...,3.5,senscritique,2885,446
1,292222,the innocents,"Wow, voilà un film qui n'a pas volé ses récomp...",4.5,senscritique,4843,755
2,292222,the innocents,Même s’il s’était essayé une première fois à l...,3.5,senscritique,5655,891
3,292222,the innocents,-Et voici donc ma palme d'or 2021 ! Le film a ...,4.5,senscritique,1178,198
4,292222,the innocents,"Eskil Vogt, réalisateur de Blind et scénariste...",3.5,senscritique,3261,526


In [29]:
# delete rows if nb_words < 3
df = df[df["nb_words"]>=3]
print("Shape :",df.shape)

Shape : (3364, 7)


In [30]:
df= preprocessing_reviews_fr(df,"contenu_complet")
df.head()

-Nombre de caractères par avis.
-Nombre de mots par avis.
-On retire la ponctuation des avis.


<ipython-input-23-4a23ebc02063>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["nb_carac_fr"] = df[review_column].str.len()#Ajout du nombre de caractères


-Tokenisation des avis.
-On retire les stop words français.
-Lemmatisation française des avis tokenisés.
-Stemmatisation française des avis tokenisés.


,id_movie,title,contenu_complet,note,plateforme,message_length,nb_words,nb_carac_fr,nb_words_fr,review_no_punct_min_fr,tokenized_fr,tokenized_no_sw_fr,lemmatized_fr,stemmed_fr
0,292222,the innocents,L’enfance a toujours été un terreau particuliè...,3.5,senscritique,2885,446,2885,446,l enfance a toujours été un terreau particuliè...,"[l, enfance, a, toujours, été, un, terreau, pa...","[enfance, a, toujours, terreau, particulièreme...","[enfance, avoir, toujours, terreau, particuliè...","[enfanc, a, toujour, terreau, particuli, ferti..."
1,292222,the innocents,"Wow, voilà un film qui n'a pas volé ses récomp...",4.5,senscritique,4843,755,4843,755,wow voilà un film qui n a pas volé ses récomp...,"[wow, voilà, un, film, qui, n, a, pas, volé, s...","[wow, voilà, film, a, volé, récompenses, derni...","[wow, voilà, film, avoir, voler, récompense, d...","[wow, voilà, film, a, vol, récompens, derni, f..."
2,292222,the innocents,Même s’il s’était essayé une première fois à l...,3.5,senscritique,5655,891,5655,891,même s il s était essayé une première fois à l...,"[même, s, il, s, était, essayé, une, première,...","[essayé, première, fois, réalisation, blind, 2...","[essayer, premier, fois, réalisation, blind, 2...","[essai, premi, fois, réalis, blind, 2014, eski..."
3,292222,the innocents,-Et voici donc ma palme d'or 2021 ! Le film a ...,4.5,senscritique,1178,198,1178,198,et voici donc ma palme d or 2021 le film a ...,"[et, voici, donc, ma, palme, d, or, 2021, le, ...","[voici, donc, palme, or, 2021, film, a, présen...","[voici, donc, palm, or, 2021, film, avoir, pré...","[voic, donc, palm, or, 2021, film, a, présent,..."
4,292222,the innocents,"Eskil Vogt, réalisateur de Blind et scénariste...",3.5,senscritique,3261,526,3261,526,eskil vogt réalisateur de blind et scénariste...,"[eskil, vogt, réalisateur, de, blind, et, scén...","[eskil, vogt, réalisateur, blind, scénariste, ...","[eskil, vogt, réalisateur, blind, scénarist, o...","[eskil, vogt, réalis, blind, scénar, oslo, 31,..."


# POS  Tagging 

In [31]:
from flair.data import Sentence
from flair.models import SequenceTagger
from IPython.display import clear_output

# Load the model
model = SequenceTagger.load("qanastek/pos-french")

/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:594: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2023-01-23 18:26:13,244 loading file /root/.flair/models/pos-french/94cd70da95da63153233ac3d01a26a100d6989e4dd7d5206949377bca7510c20.4f9987d1df0bb7f84c1e70cd8eedd9e080fc68989054f1d75b79de386ff75dc7
2023-01-23 18:26:25,456 SequenceTagger predicts: Dictionary with 69 tags: <unk>, O, DET, NFP, ADJFP, AUX, VPPMS, ADV, PREP, PDEMMS, NMS, COSUB, PINDMS, PPOBJMS, VERB, DETFS, NFS, YPFOR, VPPFS, PUNCT, DETMS, PROPN, ADJMS, PPER3FS, ADJFS, COCO, NMP, PREL, PPER1S, ADJMP, VPPMP, DINTMS, PPER3MS, PPER3MP, PREF, ADJ, DINTFS, CHIF, XFAMIL, PRELFS, SYM, NOUN, MOTINC, PINDFS, PPOBJMP, NUM, PREFP, PDEMFS, VPPFP, PPER3FP


In [32]:
def pos_tag(text) : 
    sentence= Sentence(text)
    model.predict(sentence)
    print(sentence)
    return {k : v for (k,v) in  zip(sentence.tokenized.split(),[value["value"] for value in sentence.to_dict()["all labels"]])}

In [33]:
%%capture
df["pos_tags"] =df["review_no_punct_min_fr"].apply(pos_tag)

## Filtre sur les noms

In [34]:
#Tags que l'on souhaite garder : 
keep = ["NMS","NOUN","NMP","NFS","NFP"]
def filter_keep(elem_to_keep,dict_tags): 
    return [key for (key, value) in dict_tags.items() if value in elem_to_keep]

df["nouns_tags"] =df["pos_tags"].apply(lambda dict_tags : " ".join(filter_keep(keep, dict_tags)))
df.head(3)

,id_movie,title,contenu_complet,note,plateforme,message_length,nb_words,nb_carac_fr,nb_words_fr,review_no_punct_min_fr,tokenized_fr,tokenized_no_sw_fr,lemmatized_fr,stemmed_fr,pos_tags,nouns_tags
0,292222,the innocents,L’enfance a toujours été un terreau particuliè...,3.5,senscritique,2885,446,2885,446,l enfance a toujours été un terreau particuliè...,"[l, enfance, a, toujours, été, un, terreau, pa...","[enfance, a, toujours, terreau, particulièreme...","[enfance, avoir, toujours, terreau, particuliè...","[enfanc, a, toujour, terreau, particuli, ferti...","{'l': 'DET', 'enfance': 'NFS', 'a': 'AUX', 'to...",enfance terreau épouvante protagoniste victime...
1,292222,the innocents,"Wow, voilà un film qui n'a pas volé ses récomp...",4.5,senscritique,4843,755,4843,755,wow voilà un film qui n a pas volé ses récomp...,"[wow, voilà, un, film, qui, n, a, pas, volé, s...","[wow, voilà, film, a, volé, récompenses, derni...","[wow, voilà, film, avoir, voler, récompense, d...","[wow, voilà, film, a, vol, récompens, derni, f...","{'wow': 'PROPN', 'voilà': 'VERB', 'un': 'DINTM...",film récompenses festival public critique métr...
2,292222,the innocents,Même s’il s’était essayé une première fois à l...,3.5,senscritique,5655,891,5655,891,même s il s était essayé une première fois à l...,"[même, s, il, s, était, essayé, une, première,...","[essayé, première, fois, réalisation, blind, 2...","[essayer, premier, fois, réalisation, blind, 2...","[essai, premi, fois, réalis, blind, 2014, eski...","{'même': 'ADV', 's': 'PPER3MS', 'il': 'PPER3MS...",fois réalisation travail écriture films compat...


In [35]:
ban_words = ["très", "tout", "trop", "non", "ce", "cet","cette", 'ca','ça','cela',"merci","oui"]
df["nouns_tags"] = df["nouns_tags"].apply(lambda text : [word for word in text.split() if word not in ban_words])

## Sauvegarde du dataset

In [36]:
df.to_csv(data_path+"topic_modeling_data_preprocessed.csv",index=False)

# Topic modeling 

In [37]:
!pip install pyLDAvis==2.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=284c725f9a60cae6fd36ca3a4af23e4c08d22e94daf1b33efe317fec882c5d21
  Stored in directory: /root/.cache/pip/wheels/31/8c/a0/24a443892f2134e691d59c8c6c35e19821e02f85e49871f8fd
Successfully built pyLDAvis


In [8]:
# LDA
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import pyLDAvis
# import pyLDAvis.gensim_models
from IPython.display import display
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd 

In [3]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output

    sent_topics_df[texts.name] = texts
    return(sent_topics_df)

In [5]:
def filter_noun(nouns_tags:str, list_strings) : 
    results=[]
    for noun in nouns_tags.split() : 
        for n_gram in list_strings : 
            if noun in n_gram and n_gram not in results:
                results.append(n_gram)    

    return results

In [64]:
from wordcloud import WordCloud
from matplotlib import gridspec

def generate_wordcloud(df,column_n_grams, column_cluster, value_cluster) : 
    wordcloud= WordCloud(width = 600, height=400,).generate_from_frequencies(df_results[df_results[column_cluster]==value_cluster].explode(column_n_grams).groupby(column_n_grams).agg("count").sort_values(column_cluster,ascending=False)["review_no_punct_min_fr"].to_dict())
    return wordcloud


def grid_scatter_plot_clusters_product(df,column_n_grams,column_cluster,column_treated, rate_column, n_cols=2) : 
    nb_clusters= df[column_cluster].nunique()
    n_rows= int(np.ceil(nb_clusters/n_cols))

    plt.gcf().set_dpi(300)
    fig=  plt.figure(figsize=(n_cols*10, n_rows*4+2))
    outer = gridspec.GridSpec(n_rows, n_cols,wspace=0.1, hspace=0.3)

    #colors = ["#"+''.join([random.choice('0123456789ABCDEF') for i in range(6)]) for j in range(nb_clusters)]
    list_clusters = df.groupby(column_cluster).agg("count").sort_values(column_treated,ascending=False).index.tolist()


    index_cluster= 0
    for row in range(n_rows) :
        for col in range(n_rows) : 
            if index_cluster == nb_clusters : 
                break
            
            topic= list_clusters[index_cluster]
            df_temp = df[df[column_cluster]==topic]
            df_temp2 = pd.DataFrame((df_temp.groupby(rate_column).agg("count")[column_treated]/len(df_temp))*100).reset_index().sort_values(rate_column, ascending=False)
            inner =gridspec.GridSpecFromSubplotSpec(1,2,subplot_spec=outer[index_cluster],wspace=0.05, hspace=0.1)

            #Scatter plot : 
            ax= plt.Subplot(fig,inner[0])

            sns.barplot(data=df_temp2, x=rate_column,y=column_treated, ax=ax)
            ax.set_title(f"Topic {topic}")   
            ax.set_xlabel("Note")
            ax.set_ylabel("Nombre d'avis")
            #ax.set_xticks([])
            #ax.set_yticks([])
            fig.add_subplot(ax)


            #WordCloud : 
            ax=plt.Subplot(fig,inner[1])
            
            wordcloud= generate_wordcloud(df,column_n_grams, column_cluster, topic)

            ax.imshow(wordcloud)            
            ax.axis("off")
            ax.grid(None)
            ax.set_title(f"{len(df_temp)} avis ")
            fig.add_subplot(ax)
            index_cluster+=1

    plt.tight_layout()
    plt.show()

In [56]:
df =pd.read_csv(data_path+"topic_modeling_data_preprocessed.csv")
df=df.dropna()
df["nouns_tags"]= df["nouns_tags"].apply(eval)
df["note_arrondie"] = df["note"].apply(lambda note : int(float(note.replace(",","."))))
df.head()

,id_movie,title,contenu_complet,note,plateforme,message_length,nb_words,nb_carac_fr,nb_words_fr,review_no_punct_min_fr,tokenized_fr,tokenized_no_sw_fr,lemmatized_fr,stemmed_fr,pos_tags,nouns_tags,note_arrondie
0,292222,the innocents,L’enfance a toujours été un terreau particuliè...,3.5,senscritique,2885,446,2885.0,446.0,l enfance a toujours été un terreau particuliè...,"['l', 'enfance', 'a', 'toujours', 'été', 'un',...","['enfance', 'a', 'toujours', 'terreau', 'parti...","['enfance', 'avoir', 'toujours', 'terreau', 'p...","['enfanc', 'a', 'toujour', 'terreau', 'particu...","{'l': 'DET', 'enfance': 'NFS', 'a': 'AUX', 'to...","[enfance, terreau, épouvante, protagoniste, vi...",3
1,292222,the innocents,"Wow, voilà un film qui n'a pas volé ses récomp...",4.5,senscritique,4843,755,4843.0,755.0,wow voilà un film qui n a pas volé ses récomp...,"['wow', 'voilà', 'un', 'film', 'qui', 'n', 'a'...","['wow', 'voilà', 'film', 'a', 'volé', 'récompe...","['wow', 'voilà', 'film', 'avoir', 'voler', 'ré...","['wow', 'voilà', 'film', 'a', 'vol', 'récompen...","{'wow': 'PROPN', 'voilà': 'VERB', 'un': 'DINTM...","[film, récompenses, festival, public, critique...",4
2,292222,the innocents,Même s’il s’était essayé une première fois à l...,3.5,senscritique,5655,891,5655.0,891.0,même s il s était essayé une première fois à l...,"['même', 's', 'il', 's', 'était', 'essayé', 'u...","['essayé', 'première', 'fois', 'réalisation', ...","['essayer', 'premier', 'fois', 'réalisation', ...","['essai', 'premi', 'fois', 'réalis', 'blind', ...","{'même': 'ADV', 's': 'PPER3MS', 'il': 'PPER3MS...","[fois, réalisation, travail, écriture, films, ...",3
3,292222,the innocents,-Et voici donc ma palme d'or 2021 ! Le film a ...,4.5,senscritique,1178,198,1178.0,198.0,et voici donc ma palme d or 2021 le film a ...,"['et', 'voici', 'donc', 'ma', 'palme', 'd', 'o...","['voici', 'donc', 'palme', 'or', '2021', 'film...","['voici', 'donc', 'palm', 'or', '2021', 'film'...","['voic', 'donc', 'palm', 'or', '2021', 'film',...","{'et': 'COCO', 'voici': 'VERB', 'donc': 'ADV',...","[palme, or, film, regard, compétition, cannes,...",4
4,292222,the innocents,"Eskil Vogt, réalisateur de Blind et scénariste...",3.5,senscritique,3261,526,3261.0,526.0,eskil vogt réalisateur de blind et scénariste...,"['eskil', 'vogt', 'réalisateur', 'de', 'blind'...","['eskil', 'vogt', 'réalisateur', 'blind', 'scé...","['eskil', 'vogt', 'réalisateur', 'blind', 'scé...","['eskil', 'vogt', 'réalis', 'blind', 'scénar',...","{'eskil': 'PROPN', 'vogt': 'XFAMIL', 'réalisat...","[réalisateur, scénariste, août, chapitres, cam...",3


## Topic modeling LDA 

### Premier essais pour tous les films 

In [57]:
data=df["nouns_tags"]
# Create Dictionary
id2word = corpora.Dictionary(data)
# Create Corpus
texts = data
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=15, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin ins

[(0, '0.026*"or" + 0.020*"vente" + 0.017*"navet" + 0.016*"palme" + 0.015*"distribution" + 0.015*"festival" + 0.012*"pire" + 0.011*"park" + 0.010*"hybride" + 0.009*"perspectives"'), (1, '0.059*"film" + 0.025*"histoire" + 0.021*"scénario" + 0.018*"scène" + 0.018*"scènes" + 0.017*"fois" + 0.016*"personnages" + 0.015*"films" + 0.015*"fin" + 0.013*"temps"'), (2, '0.028*"revanche" + 0.027*"racisme" + 0.019*"sommet" + 0.017*"star" + 0.014*"prostituée" + 0.013*"poing" + 0.013*"intentions" + 0.012*"maîtrise" + 0.010*"singes" + 0.009*"professeur"'), (3, '0.026*"clins" + 0.025*"espèce" + 0.022*"créatures" + 0.018*"folie" + 0.016*"vol" + 0.015*"minuit" + 0.013*"incassable" + 0.012*"descente" + 0.011*"dénouement" + 0.011*"enquête"'), (4, '0.035*"comédie" + 0.032*"prestation" + 0.023*"eau" + 0.019*"avant" + 0.016*"minute" + 0.015*"secret" + 0.015*"mec" + 0.013*"ampleur" + 0.012*"justesse" + 0.012*"dialogue"'), (5, '0.035*"opus" + 0.018*"franchise" + 0.015*"mission" + 0.015*"blockbuster" + 0.014*"cas

/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

### Mots clés

In [58]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords,nouns_tags
0,1.0,0.2894,"film, histoire, scénario, scène, scènes, fois,...","[enfance, terreau, épouvante, protagoniste, vi..."
1,7.0,0.6236,"animation, enfants, univers, fait, guerre, ave...","[film, récompenses, festival, public, critique..."
2,1.0,0.3630,"film, histoire, scénario, scène, scènes, fois,...","[fois, réalisation, travail, écriture, films, ..."
3,1.0,0.4070,"film, histoire, scénario, scène, scènes, fois,...","[palme, or, film, regard, compétition, cannes,..."
4,7.0,0.3417,"animation, enfants, univers, fait, guerre, ave...","[réalisateur, scénariste, août, chapitres, cam..."
...,...,...,...,...
3358,1.0,0.4417,"film, histoire, scénario, scène, scènes, fois,...","[scénario, facilités, clichés, message, sauveg..."
3359,1.0,0.4202,"film, histoire, scénario, scène, scènes, fois,...","[soleil, pluie, dinos, ans, façon, homme, affa..."
3360,1.0,0.5196,"film, histoire, scénario, scène, scènes, fois,...","[film, envie, déception, rendez, scénario, par..."
3361,1.0,0.4067,"film, histoire, scénario, scène, scènes, fois,...","[pays, roi, pâte, temps, réalisateur, orphelin..."


In [59]:
df_results= pd.concat([df_topic_sents_keywords.drop(columns=["nouns_tags"]), df.reset_index().drop(columns=["index"])],axis=1)
df_results.head(2)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,id_movie,title,contenu_complet,note,plateforme,message_length,nb_words,nb_carac_fr,nb_words_fr,review_no_punct_min_fr,tokenized_fr,tokenized_no_sw_fr,lemmatized_fr,stemmed_fr,pos_tags,nouns_tags,note_arrondie
0,1.0,0.2894,"film, histoire, scénario, scène, scènes, fois,...",292222,the innocents,L’enfance a toujours été un terreau particuliè...,3.5,senscritique,2885,446,2885.0,446.0,l enfance a toujours été un terreau particuliè...,"['l', 'enfance', 'a', 'toujours', 'été', 'un',...","['enfance', 'a', 'toujours', 'terreau', 'parti...","['enfance', 'avoir', 'toujours', 'terreau', 'p...","['enfanc', 'a', 'toujour', 'terreau', 'particu...","{'l': 'DET', 'enfance': 'NFS', 'a': 'AUX', 'to...","[enfance, terreau, épouvante, protagoniste, vi...",3
1,7.0,0.6236,"animation, enfants, univers, fait, guerre, ave...",292222,the innocents,"Wow, voilà un film qui n'a pas volé ses récomp...",4.5,senscritique,4843,755,4843.0,755.0,wow voilà un film qui n a pas volé ses récomp...,"['wow', 'voilà', 'un', 'film', 'qui', 'n', 'a'...","['wow', 'voilà', 'film', 'a', 'volé', 'récompe...","['wow', 'voilà', 'film', 'avoir', 'voler', 'ré...","['wow', 'voilà', 'film', 'a', 'vol', 'récompen...","{'wow': 'PROPN', 'voilà': 'VERB', 'un': 'DINTM...","[film, récompenses, festival, public, critique...",4


### Interprétation des résultats 

In [60]:
from nltk.util import ngrams
df_results["bigrams"] = df_results["review_no_punct_min_fr"].apply(lambda string : [" ".join(x) for x in list(ngrams(string.split(), 2))] )
df_results["trigrams"] = df_results["review_no_punct_min_fr"].apply(lambda string : [" ".join(x) for x in list(ngrams(string.split(), 3))] )
df_results["quadgrams"] = df_results["review_no_punct_min_fr"].apply(lambda string : [" ".join(x) for x in list(ngrams(string.split(), 4))] )
df_results.head(2)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,id_movie,title,contenu_complet,note,plateforme,message_length,nb_words,nb_carac_fr,nb_words_fr,review_no_punct_min_fr,tokenized_fr,tokenized_no_sw_fr,lemmatized_fr,stemmed_fr,pos_tags,nouns_tags,note_arrondie,bigrams,trigrams,quadgrams
0,1.0,0.2894,"film, histoire, scénario, scène, scènes, fois,...",292222,the innocents,L’enfance a toujours été un terreau particuliè...,3.5,senscritique,2885,446,2885.0,446.0,l enfance a toujours été un terreau particuliè...,"['l', 'enfance', 'a', 'toujours', 'été', 'un',...","['enfance', 'a', 'toujours', 'terreau', 'parti...","['enfance', 'avoir', 'toujours', 'terreau', 'p...","['enfanc', 'a', 'toujour', 'terreau', 'particu...","{'l': 'DET', 'enfance': 'NFS', 'a': 'AUX', 'to...","[enfance, terreau, épouvante, protagoniste, vi...",3,"[l enfance, enfance a, a toujours, toujours ét...","[l enfance a, enfance a toujours, a toujours é...","[l enfance a toujours, enfance a toujours été,..."
1,7.0,0.6236,"animation, enfants, univers, fait, guerre, ave...",292222,the innocents,"Wow, voilà un film qui n'a pas volé ses récomp...",4.5,senscritique,4843,755,4843.0,755.0,wow voilà un film qui n a pas volé ses récomp...,"['wow', 'voilà', 'un', 'film', 'qui', 'n', 'a'...","['wow', 'voilà', 'film', 'a', 'volé', 'récompe...","['wow', 'voilà', 'film', 'avoir', 'voler', 'ré...","['wow', 'voilà', 'film', 'a', 'vol', 'récompen...","{'wow': 'PROPN', 'voilà': 'VERB', 'un': 'DINTM...","[film, récompenses, festival, public, critique...",4,"[wow voilà, voilà un, un film, film qui, qui n...","[wow voilà un, voilà un film, un film qui, fil...","[wow voilà un film, voilà un film qui, un film..."


#### Filtrage des n_grams

In [62]:
columns= ["bigrams","trigrams","quadgrams"]

for col in columns  : 
    df_results[col] = df_results[~df_results["nouns_tags"].isna()].apply(lambda row : filter_noun(" ".join(row["nouns_tags"]), row[col]) , axis=1)
df_results.head(2)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,id_movie,title,contenu_complet,note,plateforme,message_length,nb_words,nb_carac_fr,nb_words_fr,review_no_punct_min_fr,tokenized_fr,tokenized_no_sw_fr,lemmatized_fr,stemmed_fr,pos_tags,nouns_tags,note_arrondie,bigrams,trigrams,quadgrams
0,1.0,0.2894,"film, histoire, scénario, scène, scènes, fois,...",292222,the innocents,L’enfance a toujours été un terreau particuliè...,3.5,senscritique,2885,446,2885.0,446.0,l enfance a toujours été un terreau particuliè...,"['l', 'enfance', 'a', 'toujours', 'été', 'un',...","['enfance', 'a', 'toujours', 'terreau', 'parti...","['enfance', 'avoir', 'toujours', 'terreau', 'p...","['enfanc', 'a', 'toujour', 'terreau', 'particu...","{'l': 'DET', 'enfance': 'NFS', 'a': 'AUX', 'to...","[enfance, terreau, épouvante, protagoniste, vi...",3,"[l enfance, enfance a, un terreau, terreau par...","[l enfance a, enfance a toujours, été un terre...","[l enfance a toujours, enfance a toujours été,..."
1,7.0,0.6236,"animation, enfants, univers, fait, guerre, ave...",292222,the innocents,"Wow, voilà un film qui n'a pas volé ses récomp...",4.5,senscritique,4843,755,4843.0,755.0,wow voilà un film qui n a pas volé ses récomp...,"['wow', 'voilà', 'un', 'film', 'qui', 'n', 'a'...","['wow', 'voilà', 'film', 'a', 'volé', 'récompe...","['wow', 'voilà', 'film', 'avoir', 'voler', 'ré...","['wow', 'voilà', 'film', 'a', 'vol', 'récompen...","{'wow': 'PROPN', 'voilà': 'VERB', 'un': 'DINTM...","[film, récompenses, festival, public, critique...",4,"[un film, film qui, le film, film de, deux fil...","[voilà un film, un film qui, film qui n, où le...","[wow voilà un film, voilà un film qui, un film..."


#### Affichage des topics 

#### Bigrams : 

In [65]:
grid_scatter_plot_clusters_product(df_results,"bigrams","Dominant_Topic","nouns_tags","note_arrondie",  n_cols=3)

Output hidden; open in https://colab.research.google.com to view.

#### Trigrams : 

In [66]:
grid_scatter_plot_clusters_product(df_results,"trigrams","Dominant_Topic","nouns_tags","note_arrondie",  n_cols=3)

Output hidden; open in https://colab.research.google.com to view.

Nous comprenons que sur tous les filsm à la fois les résultats n'ont pas trop de sens.

### On essaye à présent sur le film Interstellar

In [80]:
df= df[df["id_movie"]==df.groupby("id_movie").agg("count").sort_values("title",ascending=False).iloc[0].name]
print(f'Nous disposons de {len(df)} critiques pour Interstellar\n')
df.head(3)

Nous disposons de 120 critiques pour Interstellar



,id_movie,title,contenu_complet,note,plateforme,message_length,nb_words,nb_carac_fr,nb_words_fr,review_no_punct_min_fr,tokenized_fr,tokenized_no_sw_fr,lemmatized_fr,stemmed_fr,pos_tags,nouns_tags,note_arrondie
319,114782,interstellar,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...",5.0,senscritique,4091,664,4091.0,664.0,aïe aïe aïe nous y voilà interstellar le fi...,"['aïe', 'aïe', 'aïe', 'nous', 'y', 'voilà', 'i...","['aïe', 'aïe', 'aïe', 'voilà', 'interstellar',...","['aïe', 'aïe', 'aïe', 'voilà', 'interstellar',...","['aïe', 'aïe', 'aïe', 'voilà', 'interstellar',...","{'aïe': 'INTJ', 'nous': 'PREFP', 'y': 'PPOBJMS...","[film, nom, passions, décennie, arnaque, symbo...",5
320,114782,interstellar,"Un grand film, pour moi, c'est un film qui m'e...",4.0,senscritique,3575,611,3575.0,611.0,un grand film pour moi c est un film qui m e...,"['un', 'grand', 'film', 'pour', 'moi', 'c', 'e...","['grand', 'film', 'film', 'empêche', 'dormir',...","['grand', 'film', 'film', 'empêche', 'dormir',...","['grand', 'film', 'film', 'empêch', 'dorm', 'f...","{'un': 'DINTMS', 'grand': 'ADJMS', 'film': 'NM...","[film, crâne, esprit, genre, claque, gueule, m...",4
321,114782,interstellar,Malgré ce que j'entends dire ou lis sur le sit...,5.0,senscritique,43363,7165,43363.0,7165.0,malgré ce que j entends dire ou lis sur le sit...,"['malgré', 'ce', 'que', 'j', 'entends', 'dire'...","['malgré', 'entends', 'dire', 'lis', 'site', '...","['malgré', 'entends', 'dire', 'lire', 'site', ...","['malgr', 'entend', 'dir', 'lis', 'sit', 'aill...","{'malgré': 'PREP', 'ce': 'PDEMMS', 'que': 'COS...","[site, savoir, comparaisons, odyssée, espace, ...",5


In [84]:
data=df["nouns_tags"]
# Create Dictionary
id2word = corpora.Dictionary(data)
# Create Corpus
texts = data
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=15, random_state=100, chunksize=10, passes=10, alpha='auto', per_word_topics=True)

# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

[(0, '0.018*"b" + 0.013*"situations" + 0.012*"amateurs" + 0.012*"truc" + 0.011*"extrême" + 0.009*"visions" + 0.007*"attentes" + 0.007*"têtes" + 0.006*"distances" + 0.006*"chaos"'), (1, '0.016*"possibilité" + 0.012*"grain" + 0.009*"limite" + 0.008*"paradoxes" + 0.007*"inconnu" + 0.006*"champs" + 0.006*"distorsion" + 0.006*"incarnation" + 0.006*"arcanes" + 0.006*"nécessaire"'), (2, '0.023*"professeur" + 0.018*"fils" + 0.017*"intelligence" + 0.014*"retrouvailles" + 0.012*"équipe" + 0.011*"capacité" + 0.010*"apparition" + 0.009*"œil" + 0.009*"intérieur" + 0.009*"condition"'), (3, '0.024*"philosophie" + 0.015*"contrôle" + 0.013*"lois" + 0.010*"perception" + 0.007*"espérance" + 0.006*"nolans" + 0.006*"pelle" + 0.006*"aperçu" + 0.006*"cadrage" + 0.006*"montagnes"'), (4, '0.032*"futur" + 0.015*"ennemi" + 0.008*"dedans" + 0.007*"complot" + 0.007*"lune" + 0.007*"prof" + 0.005*"fascination" + 0.005*"cosmonaute" + 0.004*"bagarre" + 0.004*"littérature"'), (5, '0.032*"travail" + 0.023*"compositeur" 

/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [85]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords,nouns_tags
0,11.0,0.4556,"message, objet, bouche, service, décor, aventu...",NaN
1,14.0,0.5448,"bout, rebondissements, vue, moments, jeu, proj...",NaN
2,3.0,0.4298,"philosophie, contrôle, lois, perception, espér...",NaN
3,10.0,0.4222,"talent, raccourcis, décision, développement, g...",NaN
4,14.0,0.4845,"bout, rebondissements, vue, moments, jeu, proj...",NaN
...,...,...,...,...
115,8.0,0.7754,"film, espace, scénario, histoire, films, fois,...",NaN
116,8.0,0.7696,"film, espace, scénario, histoire, films, fois,...",NaN
117,8.0,0.6939,"film, espace, scénario, histoire, films, fois,...",NaN
118,8.0,0.6877,"film, espace, scénario, histoire, films, fois,...",NaN


In [88]:
df_results= pd.concat([df_topic_sents_keywords.drop(columns=["nouns_tags"]), df.reset_index().drop(columns=["index"])],axis=1)
df_results.head(2)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,id_movie,title,contenu_complet,note,plateforme,message_length,nb_words,nb_carac_fr,nb_words_fr,review_no_punct_min_fr,tokenized_fr,tokenized_no_sw_fr,lemmatized_fr,stemmed_fr,pos_tags,nouns_tags,note_arrondie
0,11.0,0.4556,"message, objet, bouche, service, décor, aventu...",114782,interstellar,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...",5.0,senscritique,4091,664,4091.0,664.0,aïe aïe aïe nous y voilà interstellar le fi...,"['aïe', 'aïe', 'aïe', 'nous', 'y', 'voilà', 'i...","['aïe', 'aïe', 'aïe', 'voilà', 'interstellar',...","['aïe', 'aïe', 'aïe', 'voilà', 'interstellar',...","['aïe', 'aïe', 'aïe', 'voilà', 'interstellar',...","{'aïe': 'INTJ', 'nous': 'PREFP', 'y': 'PPOBJMS...","[film, nom, passions, décennie, arnaque, symbo...",5
1,14.0,0.5448,"bout, rebondissements, vue, moments, jeu, proj...",114782,interstellar,"Un grand film, pour moi, c'est un film qui m'e...",4.0,senscritique,3575,611,3575.0,611.0,un grand film pour moi c est un film qui m e...,"['un', 'grand', 'film', 'pour', 'moi', 'c', 'e...","['grand', 'film', 'film', 'empêche', 'dormir',...","['grand', 'film', 'film', 'empêche', 'dormir',...","['grand', 'film', 'film', 'empêch', 'dorm', 'f...","{'un': 'DINTMS', 'grand': 'ADJMS', 'film': 'NM...","[film, crâne, esprit, genre, claque, gueule, m...",4


In [90]:
df_results["Topic_Keywords"].iloc[0]

'message, objet, bouche, service, décor, aventures, moteur, valeur, spectacle, séquence'

In [91]:
df_results["review_no_punct_min_fr"].iloc[0]

'aïe aïe aïe  nous y voilà  interstellar  le film dont on ne doit pas prononcer le nom  celui qui déchaîne les passions  film de la décennie pour certains  arnaque pour d autres  déjà moqué pour ces symboles et idées véhiculées  décrit comme melo larmoyant  ou encore mascarade de science fiction  2001 l odyssey de l espace doit figurer à chaque paragraphe  on doit bien sûr mentionner du tarkovski  ou des récits avec une notoriété moins évidente comme contact ou the black hole et enfin faire le lien avec gravity évidemment plutôt que d aller puiser dans des centaines de références et d influence diverses et variées  je considère interstellar comme le film le plus personnel de nolan  celui dans lequel il finit par assumer un message martelé depuis toujours  réalisateur régulièrement critiqué pour sa froideur  sa prétention ou son ambition mal placée  il est pourtant à mon sens un grand romantique méconnu  l amour est un sentiment qui transpire chaque parcelle de ses films  mémento réputé